In [1]:
import tensorflow as tf
import get
import pandas as pd
import numpy as np
#from progressbar import ProgressBar
#pbar = ProgressBar()
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
import dask.dataframe as dd
from dask.distributed import Client, progress
import os
client = Client(n_workers=2, threads_per_worker=2, memory_limit='8GB')
client

Client Scheduler: tcp://127.0.0.1:49926 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 16.00 GB


# Getting Tickers

So, for some reason, I can't find a nice downloadable csv file that already has all of the tickers in the nyse. I did, however, find a website that can be used to scrape off of!
So first, lets get all the tickers we are going to use

In [2]:
os.system("curl --ftp-ssl anonymous:jupi@jupi.com "
          "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt "
          "> nasdaq.lst")


0

In [3]:
#Reading from that file
with open('nasdaq.lst', 'r') as f:
    file = f.read()
find_string = file.split('</html>')
ticker_string = find_string[len(find_string)-1]
rows = ticker_string.split('\n')
tickers = [row.split('|')[0] for row in rows][2:]

In [4]:
data = dict()
for t in tickers:
   # try:
    print(f'----{t}----')
    price_data = get.daily(t)
    print(price_data)
    news_data = get.News(t)
    data.update({t:[price_data, news_data]})
    print('------------')
   # except AttributeError:
    #    pass
#In case this expirement needs to be replicated
# import pickle
# with open('stockReg.pkl', 'wb') as f:
#     pickle.dump(data, f)    

----AACG----
     timestamp    open    high    low   close  adjusted_close  volume  \
0   2020-09-10  1.0800  1.0900  1.070  1.0700          1.0700   26856   
1   2020-09-09  1.1100  1.1400  1.070  1.0900          1.0900    9963   
2   2020-09-08  1.0900  1.1400  1.070  1.1400          1.1400   17481   
3   2020-09-04  1.0900  1.1100  1.068  1.1000          1.1000   27247   
4   2020-09-03  1.1320  1.1500  1.060  1.1100          1.1100   32074   
..         ...     ...     ...    ...     ...             ...     ...   
95  2020-04-27  0.7500  0.7877  0.612  0.6730          0.6730   62808   
96  2020-04-24  0.8102  0.8102  0.750  0.7500          0.7500    7119   
97  2020-04-23  0.7768  0.7780  0.735  0.7400          0.7400    3908   
98  2020-04-22  0.7700  0.7799  0.750  0.7640          0.7640   16506   
99  2020-04-21  0.7800  0.7801  0.702  0.7702          0.7702   10304   

    dividend_amount  split_coefficient  
0               0.0                1.0  
1               0.0         

C:\VApps\Anaconda3\envs\tf\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


------------
----AACQ----
    timestamp   open    high    low  close  adjusted_close   volume  \
0  2020-09-10  10.00  10.025  9.836   9.96            9.96  1758528   
1  2020-09-09  10.00  10.010  9.910   9.99            9.99  1775622   
2  2020-09-08  10.27  10.270  9.915   9.95            9.95    82469   
3  2020-09-04  10.00  10.000  9.950  10.00           10.00    11506   

   dividend_amount  split_coefficient  
0              0.0                1.0  
1              0.0                1.0  
2              0.0                1.0  
3              0.0                1.0  


AttributeError: 'NoneType' object has no attribute 'find_all'

# I haven't finished working on data processing/scraping for this one. Below is the idea I had for using ML to determine price direction from market data